In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data_set.csv')

In [3]:
df.head()
df = df.drop(['title', 'ai_generated'], axis=1)

In [4]:
df

,abstract,label
0,Advanced electromagnetic potentials are indi...,0
1,This research paper investigates the question ...,1
2,We give an algorithm for finding network enc...,0
3,The paper presents an efficient centralized bi...,1
4,We introduce an exponential random graph mod...,0
...,...,...
4048,This research paper investigates the vortex dy...,1
4049,Given a remarkable representation of the gen...,0
4050,The Veldkamp space of two-qubits is a mathemat...,1
4051,The equilibration of macroscopic degrees of ...,0


In [ ]:
# x = list(df['abstract'])
# y = list(df['label'])

In [ ]:
# ## Train test split
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [11]:
from datasets import Dataset

In [ ]:
df = Dataset.from_pandas(df)

In [20]:
df

Dataset({
    features: ['abstract', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4053
})

In [13]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [14]:
def tokenize(examples):
    return tokenizer(examples["abstract"], padding="max_length", truncation=True)

df = df.map(tokenize)

Map:   0%|          | 0/4053 [00:00<?, ? examples/s]

Map: 100%|██████████| 4053/4053 [00:03<00:00, 1281.39 examples/s]


In [15]:
df

Dataset({
    features: ['abstract', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4053
})

In [17]:
train_test = df.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test['train']
test_dataset = train_test['test']

In [19]:
train_dataset

Dataset({
    features: ['abstract', 'label', 'input_ids', 'attention_mask'],
    num_rows: 3242
})

In [11]:
train_encodings = tokenizer(x_train, truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

In [12]:
import tensorflow as tf
# !pip install tensorflow==2.15.0


In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [14]:
from tensorflow.keras.optimizers import Adam

In [15]:
from transformers import TFDistilBertForSequenceClassification

In [16]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels =2 )

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were n

In [17]:
model.compile(optimizer='adam')

In [18]:
model.fit(
    x = train_dataset,
    validation_data = test_dataset,
    epochs = 2
)

Epoch 1/2
   7/3242 [..............................] - ETA: 6:01:20 - loss: 0.8665

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/my_model.h5')